# Online Retail II Data Set

In this notebook we will prepare and store the Online Retail II Data Set stored on the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Online+Retail+II)


**Citation:**

Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

## Download the data

- Navigate to the [data folder](https://archive.ics.uci.edu/ml/machine-learning-databases/00502/).
- Download the file called **online_retail_II.xlsx**.
- Save the Excel file into the **datasets** folder at the root of this repository.

In [1]:
import pandas as pd

# Load data

In [2]:
# If you downloaded and stored the file as explained
# above, it should be located here:

file = "../Datasets/online_retail_II.xlsx"

In [3]:
# The data is provided as two sheets in a single Excel file.
# Each sheet contains a different time period.
# Load both and join into a single dataframe.

df_1 = pd.read_excel(file, sheet_name="Year 2009-2010")
df_2 = pd.read_excel(file, sheet_name="Year 2010-2011")

df = pd.concat([df_1, df_2])

In [4]:
# Inspect dataframe

df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [5]:
# Rename columns

df.columns = [
    "invoice",
    "stock_code",
    "description",
    "quantity",
    "invoice_date",
    "price",
    "customer_id",
    "country",
]

# Process data

Remove null customer ids.

In [6]:
mask = ~df["customer_id"].isnull()
df = df[mask]

Create a flag for when an order is cancelled. Cancelled orders contain 
the letter `C` at the start of the invoice.

In [7]:
df["is_cancelled"] = df["invoice"].apply(lambda x: str(x)[0] == "C")

Remove transactions which are negative quantities sold and are not cancelled orders.

In [8]:
mask = ~(~df["is_cancelled"] & df["quantity"] < 0)

df = df[mask]

Compute revenue.

In [9]:
df["revenue"] = df["quantity"] * df["price"]

To compute gross revenue and quantity sold we filter out cancelled orders.

After this, we resample the data at a weekly level.

In [10]:
mask = ~df["is_cancelled"]

df_gross = (
    df.loc[mask, ["invoice_date", "quantity", "revenue", "country"]]
    .groupby("country")
    .resample("W", on="invoice_date")
    .sum(numeric_only=True)
)

In [11]:
df_gross.index.rename(["country", "week"], inplace=True)

# Save data

We will save 3 different versions of the preprocessed dataset for different demos.

## Weekly sampled

In [12]:
df_gross_countries = df_gross.reset_index(level="country")

In [13]:
countries = [
    'United Kingdom',
    'Belgium',
    "EIRE",
    'Germany',
    "France",
    'Spain',
]

In [14]:
df_gross_countries[df_gross_countries["country"].isin(countries)].to_csv(
    "../Datasets/online_retail_dataset_countries.csv",
    index=True,
)

## Unstacked countries

In [15]:
y = df_gross.unstack("country")["revenue"]

In [16]:
y.head()

country,Australia,Austria,Bahrain,Belgium,Brazil,Canada,Channel Islands,Cyprus,Czech Republic,Denmark,...,Singapore,Spain,Sweden,Switzerland,Thailand,USA,United Arab Emirates,United Kingdom,Unspecified,West Indies
week,,,,,,,,,,,,,,,,,,,,,
2009-12-06,196.1,NaN,NaN,439.1,NaN,NaN,989.18,760.69,NaN,1008.00,...,NaN,435.88,NaN,NaN,NaN,141.0,NaN,213000.35,NaN,NaN
2009-12-13,0.0,1429.83,NaN,8.5,NaN,NaN,0.00,0.00,NaN,0.00,...,NaN,412.60,285.3,NaN,NaN,0.0,517.7,195810.04,NaN,NaN
2009-12-20,75.0,0.00,NaN,0.0,NaN,NaN,0.00,2796.29,NaN,429.66,...,NaN,1952.64,0.0,589.4,NaN,0.0,0.0,182396.74,NaN,NaN
2009-12-27,0.0,568.51,NaN,0.0,NaN,NaN,0.00,0.00,NaN,0.00,...,NaN,5149.06,0.0,0.0,NaN,0.0,0.0,22007.77,NaN,NaN
2010-01-03,0.0,0.00,NaN,0.0,NaN,NaN,0.00,0.00,NaN,0.00,...,NaN,0.00,0.0,0.0,NaN,0.0,0.0,0.00,NaN,NaN


In [17]:
y.to_csv("../Datasets/online_retail_dataset.csv")

## Raw data

In [18]:
# columns needed for demo
cols = ["invoice_date", "description", "revenue"]

# just UK
df = df[df["country"] == "United Kingdom"]

# save
df[cols].to_csv("../Datasets/online_retail_dataset_all.csv", index=False)